<a href="https://colab.research.google.com/github/acl-jianghao/Langchain-demos/blob/main/07_sequentialChain_tools_agent.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [35]:
!pip install -Uqq openai langchain tiktoken colab-env pymupdf faiss-cpu google-search-results
!pip install -Uqq pyspark-ai pyspark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 310.8/310.8 MB 963.3 kB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.0/143.0 kB 13.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.5/11.5 MB 41.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 47.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.3/12.3 MB 39.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 50.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.6/62.6 kB 6.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 341.8/341.8 kB 29.9 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
ipython 7.34.0 requires jedi>=0.16, which is not installed.
google-colab 1.0.0 requires pandas==1.5.3, but you have pandas 2.0.3 which is inc

In [37]:
import os
import colab_env

In [ ]:
# from langchain.agents import load_tools

# tools = load_tools(["serpapi"])
# tools.append()

In [10]:
# from langchain.utilities import SerpAPIWrapper

# search = SerpAPIWrapper()

# search.run("what is the weather of Hangzhou of next 7 days?")

'Precipitation: Rain: 0.01 Snow: 0. Precipitation Chance: 11%. E. Wind: 7 mph. Jul 24 – Jul 30Jul 31 – Aug 6Aug 7 – Aug 13. See weather overview ...'

In [38]:
import openai

openai.api_key = os.getenv('OPENAI_API_KEY')
serpapi_api_key = os.getenv('SERPAPI_API_KEY')

In [107]:
# from langchain.chat_models import ChatOpenAI
from langchain.llms import OpenAI
from langchain.agents import load_tools, initialize_agent


# llm=ChatOpenAI(temperature=0, verbose=True)
llm=OpenAI(temperature=0, verbose=True)
# tools = load_tools(["serpapi"], llm, serpapi_api_key=serpapi_api_key)
# agent = initialize_agent(tools, llm, agent="zero-shot-react-description", verbose=True)

# result = agent.run("please list the weather of each day in Hangzhou of the next 7 days?")

In [108]:
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain, SequentialChain


# Prompt templates
serp_template = PromptTemplate(
    input_variables = ['city'],
    template="""Please list the weather, temperature and air quality of each day in {city} of
            the next 7 days and response should be a string in code block"""
)

tell_template = PromptTemplate(
    input_variables = ['string'],
    template="""Tell me the average temperature in {string},
            the answer should only have the average temperature in Chinese """
)



In [109]:
weather_chain = LLMChain(llm=llm, prompt=serp_template, verbose=False, output_key='string')
tell_chain = LLMChain(llm=llm, prompt=tell_template, verbose=False, output_key='average_temperature')


sequential_chain = SequentialChain(
    chains = [weather_chain, tell_chain],
    input_variables=['city'],
    output_variables=['string', 'average_temperature'],
    verbose=True)

# sequential_chain = SequentialChain(
#     chains = [weather_chain],
#     input_variables=['city'],
#     output_variables=['json'],
#     verbose=True)

In [110]:
sequential_chain('Hangzhou')



> Entering new SequentialChain chain...

> Finished chain.


{'city': 'Hangzhou',
 'string': '\n\nDay 1: Weather: Sunny, Temperature: 25°C, Air Quality: Good\nDay 2: Weather: Cloudy, Temperature: 22°C, Air Quality: Moderate\nDay 3: Weather: Rainy, Temperature: 20°C, Air Quality: Poor\nDay 4: Weather: Partly Cloudy, Temperature: 23°C, Air Quality: Good\nDay 5: Weather: Sunny, Temperature: 25°C, Air Quality: Moderate\nDay 6: Weather: Cloudy, Temperature: 22°C, Air Quality: Poor\nDay 7: Weather: Partly Cloudy, Temperature: 24°C, Air Quality: Good',
 'average_temperature': '\n\n平均温度：22.7°C'}

In [57]:
# from pyspark_ai import SparkAI

# spark_ai = SparkAI(llm=llm, verbose=True)

# spark_ai.activate()


tools + agent

In [111]:
from langchain.agents import load_tools, initialize_agent

tools = load_tools(["serpapi"], llm, serpapi_api_key=serpapi_api_key)


# agent = initialize_agent(tools, llm, agent="zero-shot-react-description", verbose=True)

# result = agent.run("please list the weather of each day in Hangzhou of the next 7 days?")

In [ ]:
from langchain.tools import BaseTool

class CalculateAverage(BaseTool):
    name = "Calculate Average"
    description = 'use this tool to calculate the average value.'

    def _run(self, expr: str):
        return 10.1

    def _arun(self, query: str):
        raise NotImplementedError("Async function is not ready yet")

# tools.append(CalculateAverage())


tools

In [123]:
agent = initialize_agent(
    tools,
    llm,
    agent="zero-shot-react-description",
    verbose=True
)

agent.run("please tell me the temprature average of Hangzhou in the next 7 days?")

# agent = initialize_agent(tools, llm, agent="zero-shot-react-description", verbose=True)

# agent.run("please list the weather of each day in Hangzhou of the next 7 days?")



> Entering new AgentExecutor chain...
 I need to find the average temperature of Hangzhou in the next 7 days
Action: Search
Action Input: "Hangzhou temperature forecast 7 days"
Observation: 2 Week Extended Forecast in Hangzhou, Zhejiang, China ; 85 / 78 °F · 85 / 78 °F · 87 / 79 °F ...
Thought: I need to calculate the average temperature
Action: Calculate Average
Action Input: 85 / 78 °F · 85 / 78 °F · 87 / 79 °F ...
Observation: 10.1
Thought: I now know the final answer
Final Answer: The average temperature of Hangzhou in the next 7 days is 10.1°F.

> Finished chain.


'The average temperature of Hangzhou in the next 7 days is 10.1°F.'

gmail toolkit: https://python.langchain.com/docs/integrations/toolkits/gmail